In [1]:
!pip install -qU  langchain-core==0.2.9
!pip install -qU langchain-community==0.2.5
!pip install -qU  'crewai[tools]'==0.32.0
!pip install -qU langchain-groq==0.1.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-anthropic 0.1.23 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.2.9 which is incompatible.
langchain-google-vertexai 1.0.10 requires langchain-core<0.3,>=0.2.33, but you have langchain-core 0.2.9 which is incompatible.
langchain-openai 0.1.22 requires langchain-core<0.3.0,>=0.2.33, but you have langchain-core 0.2.9 which is incompatible.
langchain-google-genai 1.0.10 requires langchain-core<0.3,>=0.2.33, but you have langchain-core 0.2.9 which is incompatible.
langchain-groq 0.1.9 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.2.9 which is incompatible.
langchain-community 0.2.16 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.2.9 which is incompatible.
langchain-aws 0.1.18 requires langchain-core<0.3,>=0.2.33, but you have langchain-cor

In [2]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

/Users/aaronsantiagopedrazacardenas/opt/anaconda3/envs/stream/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("/Users/aaronsantiagopedrazacardenas/Desktop/github-version1/Agent_BDG/datos_sinteticos/analisis.csv")
df.head()

,id_interaccion,id_cliente,id_producto,fecha_interaccion,canal,aceptado,segmento
0,INT_000001,CLI_000673,PRD_0026,2025-08-24,Sucursal,False,Premium
1,INT_000002,CLI_000758,PRD_0003,2025-04-09,Digital,False,Classic
2,INT_000003,CLI_000268,PRD_0029,2025-02-22,App Móvil,True,Basic
3,INT_000004,CLI_000621,PRD_0008,2025-08-26,Digital,True,Premium
4,INT_000005,CLI_000380,PRD_0010,2024-11-23,Sucursal,True,Classic


In [ ]:
connection = sqlite3.connect("analisis.db")
df.to_sql(name="analisis", con=connection)

In [ ]:
llm = ChatGroq(
    temperature=0,
    api_key="",
    #model_name="llama3-70b-8192",
    model_name="gemma2-9b-it"
)

In [85]:
db = SQLDatabase.from_uri("sqlite:///analisis.db")


In [86]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

Using Tool: list_tables


'analisis'

In [87]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

print(tables_schema.run("analisis"))

Using Tool: tables_schema

CREATE TABLE analisis (
	"index" INTEGER, 
	id_interaccion TEXT, 
	id_cliente TEXT, 
	id_producto TEXT, 
	fecha_interaccion TEXT, 
	canal TEXT, 
	aceptado INTEGER, 
	segmento TEXT
)

/*
3 rows from analisis table:
index	id_interaccion	id_cliente	id_producto	fecha_interaccion	canal	aceptado	segmento
0	INT_000001	CLI_000673	PRD_0026	2025-08-24	Sucursal	0	Premium
1	INT_000002	CLI_000758	PRD_0003	2025-04-09	Digital	0	Classic
2	INT_000003	CLI_000268	PRD_0029	2025-02-22	App Móvil	1	Basic
*/


In [88]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("SELECT * FROM analisis LIMIT 5")

Using Tool: execute_sql


"[(0, 'INT_000001', 'CLI_000673', 'PRD_0026', '2025-08-24', 'Sucursal', 0, 'Premium'), (1, 'INT_000002', 'CLI_000758', 'PRD_0003', '2025-04-09', 'Digital', 0, 'Classic'), (2, 'INT_000003', 'CLI_000268', 'PRD_0029', '2025-02-22', 'App Móvil', 1, 'Basic'), (3, 'INT_000004', 'CLI_000621', 'PRD_0008', '2025-08-26', 'Digital', 1, 'Premium'), (4, 'INT_000005', 'CLI_000380', 'PRD_0010', '2024-11-23', 'Sucursal', 1, 'Classic')]"

In [89]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

#check_sql.run("SELECT * LIMIT 5 table = interacciones")

In [90]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [91]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

In [92]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)

In [93]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [94]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)


In [95]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

In [96]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=2,
    memory=False,
    output_log_file="crew.log",
)

2025-10-06 18:54:42,609 - 140704469618560 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [97]:
inputs = {
    "query": "¿Qué segmento tiene mayor probabilidad de aceptar Tarjetas de Crédito?"
}

result = crew.kickoff(inputs=inputs)

 [2025-10-06 18:54:42][DEBUG]: == Working Agent: Senior Database Developer
 [2025-10-06 18:54:42][INFO]: == Starting Task: Extract data that is required for the query ¿Qué segmento tiene mayor probabilidad de aceptar Tarjetas de Crédito?.


 

analisis

 


CREATE TABLE analisis (
	"index" INTEGER, 
	id_interaccion TEXT, 
	id_cliente TEXT, 
	id_producto TEXT, 
	fecha_interaccion TEXT, 
	canal TEXT, 
	aceptado INTEGER, 
	segmento TEXT
)

/*
3 rows from analisis table:
index	id_interaccion	id_cliente	id_producto	fecha_interaccion	canal	aceptado	segmento
0	INT_000001	CLI_000673	PRD_0026	2025-08-24	Sucursal	0	Premium
1	INT_000002	CLI_000758	PRD_0003	2025-04-09	Digital	0	Classic
2	INT_000003	CLI_000268	PRD_0029	2025-02-22	App Móvil	1	Basic
*/

 

```sql
SELECT segmento, AVG(aceptado) AS promedio_aceptado FROM analisis GROUP BY segmento ORDER BY promedio_aceptado DESC
```

 

[('Premium', 0.5976627712854758), ('Classic', 0.434009102192801), ('Basic', 0.2967509025270758)]

 [2025-10-06 18:54:45][DEBUG]: == [Senior Database Developer] Task output: [('Premium', 0.5976627712854758), ('Classic', 0.434009102192801), ('Basic', 0.2967509025270758)]


 [2025-10-06 18:54:45][DEBUG]: == Working Agent: Senior Data Analyst
 [2025-10-06 18:5

In [98]:
inputs = {
    "query": "¿Cuál canal funciona mejor para Classic?”"
}

result = crew.kickoff(inputs=inputs)

 [2025-10-06 18:54:46][DEBUG]: == Working Agent: Senior Database Developer
 [2025-10-06 18:54:46][INFO]: == Starting Task: Extract data that is required for the query ¿Cuál canal funciona mejor para Classic?”.
 

analisis

 


CREATE TABLE analisis (
	"index" INTEGER, 
	id_interaccion TEXT, 
	id_cliente TEXT, 
	id_producto TEXT, 
	fecha_interaccion TEXT, 
	canal TEXT, 
	aceptado INTEGER, 
	segmento TEXT
)

/*
3 rows from analisis table:
index	id_interaccion	id_cliente	id_producto	fecha_interaccion	canal	aceptado	segmento
0	INT_000001	CLI_000673	PRD_0026	2025-08-24	Sucursal	0	Premium
1	INT_000002	CLI_000758	PRD_0003	2025-04-09	Digital	0	Classic
2	INT_000003	CLI_000268	PRD_0029	2025-02-22	App Móvil	1	Basic
*/

 

```sql
SELECT canal, COUNT(*) AS cantidad FROM analisis WHERE segmento = 'Classic' GROUP BY canal ORDER BY cantidad DESC LIMIT 1;
```

 

[('App Móvil', 656)]

 [2025-10-06 18:54:49][DEBUG]: == [Senior Database Developer] Task output: [('App Móvil', 656)]


 [2025-10-06 18:54:49

In [99]:
inputs = {
    "query": "¿Dame los top clientes con oportunidad para aceptar productos."
}

result = crew.kickoff(inputs=inputs)

 [2025-10-06 18:54:50][DEBUG]: == Working Agent: Senior Database Developer
 [2025-10-06 18:54:50][INFO]: == Starting Task: Extract data that is required for the query ¿Dame los top clientes con oportunidad para aceptar productos..
 

analisis

 


CREATE TABLE analisis (
	"index" INTEGER, 
	id_interaccion TEXT, 
	id_cliente TEXT, 
	id_producto TEXT, 
	fecha_interaccion TEXT, 
	canal TEXT, 
	aceptado INTEGER, 
	segmento TEXT
)

/*
3 rows from analisis table:
index	id_interaccion	id_cliente	id_producto	fecha_interaccion	canal	aceptado	segmento
0	INT_000001	CLI_000673	PRD_0026	2025-08-24	Sucursal	0	Premium
1	INT_000002	CLI_000758	PRD_0003	2025-04-09	Digital	0	Classic
2	INT_000003	CLI_000268	PRD_0029	2025-02-22	App Móvil	1	Basic
*/

 

```sql
SELECT id_cliente, COUNT(*) AS oportunidades FROM analisis WHERE aceptado = 1 GROUP BY id_cliente ORDER BY oportunidades DESC LIMIT 10;
```

 [2025-10-06 18:54:52][DEBUG]: == [Senior Database Developer] Task output: ```
id_cliente	oportunidades
CLI_00